In [1]:
import mlflow
import mlflow.tensorflow
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping


Настройка MLflow

In [2]:
data_name = "ex8_X_all_for_y_2,3,13"
model_name = "LSTM_"
run_name =  model_name + data_name + "_v.10"

# установка пути к хранилищу Mlflow
mlflow.set_tracking_uri("file:///E:/Документы/Data Science/Projects/Equipment-Failure-Prediction/mlruns")
mlflow.set_experiment("Equipment-Failure-Prediction_M3")

def run_mlflow():
    '''Function making one run in current experiment'''

    mlflow.start_run(run_name=run_name)
    mlflow.log_param("X_columns", "X_all_columns")
    mlflow.log_param("y_columns", y.columns)
    mlflow.log_param("split_date", date)
    mlflow.log_param("shuffle_in_model.fit", shuffle)


In [3]:
#mlflow.end_run()
#mlflow.active_run()

In [4]:
#ЭКСГАУСТЕР №8
X = pd.read_parquet('../../data/processed/ex8_M3_X_all_for_y_2,3,13.parquet')
y = pd.read_parquet('../../data/processed/ex8_M3_y_2,3,13.parquet')

In [5]:
X.head(3)

,ЭКСГАУСТЕР 8. ТОК РОТОРА 1,ЭКСГАУСТЕР 8. ТОК РОТОРА 2,ЭКСГАУСТЕР 8. ТОК СТАТОРА,ЭКСГАУСТЕР 8. ДАВЛЕНИЕ МАСЛА В СИСТЕМЕ,ЭКСГАУСТЕР 8. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 1,ЭКСГАУСТЕР 8. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 2,ЭКСГАУСТЕР 8. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 3,ЭКСГАУСТЕР 8. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 4,ЭКСГАУСТЕР 8. ТЕМПЕРАТУРА МАСЛА В СИСТЕМЕ,ЭКСГАУСТЕР 8. ТЕМПЕРАТУРА МАСЛА В МАСЛОБЛОКЕ,ЭКСГАУСТЕР 8. ВИБРАЦИЯ НА ОПОРЕ 1,ЭКСГАУСТЕР 8. ВИБРАЦИЯ НА ОПОРЕ 2,ЭКСГАУСТЕР 8. ВИБРАЦИЯ НА ОПОРЕ 3,ЭКСГАУСТЕР 8. ВИБРАЦИЯ НА ОПОРЕ 3. ПРОДОЛЬНАЯ.,ЭКСГАУСТЕР 8. ВИБРАЦИЯ НА ОПОРЕ 4,ЭКСГАУСТЕР 8. ВИБРАЦИЯ НА ОПОРЕ 4. ПРОДОЛЬНАЯ.
DT,,,,,,,,,,,,,,,,
2019-01-16 13:21:00,1.45,1.45,0.0,4.940000,29.210000,29.210000,28.390000,28.80,23.530000,25.960000,0.100000,0.070000,0.030000,0.270000,0.060000,0.300000
2019-01-16 13:21:10,1.74,1.74,0.0,5.130000,28.881099,29.367858,28.962225,27.58,23.283353,26.122788,0.085069,0.053819,0.034722,0.228299,0.050347,0.302083
2019-01-16 13:21:20,1.16,1.16,0.0,5.283333,29.115464,29.476026,28.935183,28.39,23.436592,26.411239,0.075231,0.045332,0.031829,0.227623,0.053048,0.289352


In [6]:
y.columns

Index(['Y_ЭКСГАУСТЕР А/М №8_КОРПУС ЭКСГ. №8',
       'Y_ЭКСГАУСТЕР А/М №8_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ. №8',
       'Y_ЭКСГАУСТЕР А/М №8_КОЖУХ МУФТЫ ЭКСГ. №8'],
      dtype='object')

In [7]:
print(X.shape)
print(y.shape)

(5278618, 16)
(5278618, 3)


In [8]:
# Масштабирование  и разделение
scaler = MinMaxScaler()
X_norm = scaler.fit_transform(X)

date = '2020-04-01 08:00:00'
split_date = X.index.get_loc(date)

X_train = X_norm[:split_date]
y_train = y[:split_date]
X_val = X_norm[split_date:]
y_val = y[split_date:]


In [10]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(3807844, 16)
(3807844, 3)
(1470774, 16)
(1470774, 3)


In [11]:
# reshape for input 
timesteps = 1
samples_train = int(np.floor(X_train.shape[0]/timesteps))
samples_val = int(np.floor(X_val.shape[0]/timesteps))

X_train = X_train.reshape(samples_train, timesteps, X_train.shape[1])   #samples, timesteps, sensors
X_val = X_val.reshape(samples_val, timesteps, X_val.shape[1])

print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(3807844, 1, 16)
(3807844, 3)
(1470774, 1, 16)
(1470774, 3)


In [12]:
del X, X_norm

In [13]:
out = y_train.shape[1]
shuffle = False
batch_size = 32
early_stop = EarlyStopping(monitor='val_loss', patience=5)

In [14]:
tf.keras.backend.clear_session()  # очищаем предыдушие следы модели

In [15]:
# Модель
model = tf.keras.Sequential([
    layers.LSTM(64, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    layers.LSTM(64, activation='relu'),
    #layers.Dropout(0.2),
    #layers.Dense(64),
    layers.Dense(out, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 64)             20736     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 3)                 195       
                                                                 
Total params: 53,955
Trainable params: 53,955
Non-trainable params: 0
_________________________________________________________________


In [16]:
run_mlflow()

# Обучение на объединенной выборке
history = model.fit(X_train, y_train,
                    epochs=10,
                    #batch_size=batch_size,
                    shuffle=shuffle,  #False
                    validation_data=(X_val, y_val),
                    callbacks=[early_stop])

Epoch 1/10
118996/118996 [==============================] - 369s 3ms/step - loss: 0.0111 - accuracy: 0.1912 - val_loss: 11.5743 - val_accuracy: 0.1096
Epoch 2/10
118996/118996 [==============================] - 371s 3ms/step - loss: 0.0128 - accuracy: 0.6173 - val_loss: 13.7067 - val_accuracy: 0.1096
Epoch 3/10
118996/118996 [==============================] - 373s 3ms/step - loss: 0.0125 - accuracy: 0.6029 - val_loss: 12.1421 - val_accuracy: 0.1096
Epoch 4/10
118996/118996 [==============================] - 377s 3ms/step - loss: 0.0102 - accuracy: 0.1574 - val_loss: 8.7236 - val_accuracy: 0.1096
Epoch 5/10
118996/118996 [==============================] - 382s 3ms/step - loss: 0.0076 - accuracy: 0.5890 - val_loss: 12.4457 - val_accuracy: 0.1096
Epoch 6/10
118996/118996 [==============================] - 382s 3ms/step - loss: 0.0089 - accuracy: 0.1912 - val_loss: 9.7537 - val_accuracy: 0.1096
Epoch 7/10
118996/118996 [==============================] - 390s 3ms/step - loss: 0.0074 - accur

Результаты прогноза

In [17]:
y_pred = model.predict(X_val)
y_pred = np.round(y_pred)

45962/45962 [==============================] - 63s 1ms/step


In [18]:
accuracy = accuracy_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred, average='weighted')

TP = ((y_pred == 1) * (y_val == 1)).sum()
FP = ((y_pred == 1) * (y_val != 1)).sum()
FN = ((y_pred != 1) * (y_val == 1)).sum()
J = TP / (TP + FP + FN)

print('accuracy:', accuracy)
print('f1-score:', f1)
print('J-index', J)

accuracy: 0.19774826043974125
f1-score: 0.862032207193632
J-index Y_ЭКСГАУСТЕР А/М №8_КОРПУС ЭКСГ. №8                            0.720734
Y_ЭКСГАУСТЕР А/М №8_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ. №8    0.307388
Y_ЭКСГАУСТЕР А/М №8_КОЖУХ МУФТЫ ЭКСГ. №8                       1.000000
dtype: float64


In [19]:
mlflow.log_metric("accuracy", accuracy)
mlflow.log_metric("f1-score", f1)
mlflow.log_metric("j-index", J.mean())
#mlflow.tensorflow.log_model(model, "model")
mlflow.end_run()